In [1]:
import pandas as pd
import csv
from bs4 import BeautifulSoup
import requests
from os import walk, makedirs
import time
import random
import zipfile
import json
import numpy as np
import warnings
warnings.filterwarnings('ignore')

Extract the list of genes to download from gene@home

In [2]:
gene_list = []

gene_list = pd.read_csv("./dataset/to_scrape.txt", sep=', ')

gene_list = list(gene_list.columns)

fantom_db = pd.read_csv("./dataset/anno-fantom.csv")
snoDB = pd.read_csv("./dataset/snoDB_All_V2.0.csv",keep_default_na=False)

""" gene_new = {"SNORA11", "SNORA74A", "SNORA63C", "SNORD50B", "SNORD89", "SNORA74A", "SNORD57", "SNORD7", "SNORA31", "SNORD65", "SNORD6", "SNORA25", "SNORA32", "SNORA51", "SNORD8", "SNORA63", "SNORA4", "SNORA66", "SNORA20", "SNORA20", "SNORA67", "SNORD22", "SNORD58C", "SNORD108"}

gene_list = set(gene_list).intersection(gene_new) """

' gene_new = {"SNORA11", "SNORA74A", "SNORA63C", "SNORD50B", "SNORD89", "SNORA74A", "SNORD57", "SNORD7", "SNORA31", "SNORD65", "SNORD6", "SNORA25", "SNORA32", "SNORA51", "SNORD8", "SNORA63", "SNORA4", "SNORA66", "SNORA20", "SNORA20", "SNORA67", "SNORD22", "SNORD58C", "SNORD108"}\n\ngene_list = set(gene_list).intersection(gene_new) '

Download all the genes

In [4]:
def get_gene_links(gene_list):
    url = 'https://gene.disi.unitn.it/test/gene_history-z.php'
    data = {
        'org': 'Hs+(Homo+sapiens)',
        'tsize': '',
        'iter': '',
        'alpha': '0.05',
        'lgn': '',
        'submit': 'SEARCH'}
    missing_list = []
    isoforms = {}

    for gene in gene_list:
        isoforms[gene] = []
        print(f"Looking for {gene}")
        data['lgn'] = gene
        soup = BeautifulSoup(requests.post(url, data = data).text, "lxml")

        if soup.find("div",{"id":"dialog-end"}) != None:
            missing_list.append(gene)
        else:
            table = soup.find("tbody")
            rows = table.find_all("tr")
            for row in rows:
                columns = row.find_all("td")
                if columns[2].string.split("-")[1] == gene:
                    isoforms[gene].append(columns[0].string)
                    downloaded_file = requests.get(f"https://gene.disi.unitn.it/test/gene_result-z.php?id={columns[0].string}_{columns[1].string}", allow_redirects=True)
                    open(f'./gene_expansions/{columns[0].string}_{columns[1].string}.zip', 'wb').write(downloaded_file.content) 
        time.sleep(random.uniform(1.5,2.0))

    return missing_list, isoforms

makedirs("./gene_expansions", exist_ok=True)
missing_list, isoforms = get_gene_links(gene_list)
print(f"Missing genes: {', '.join(missing_list)}")

Looking for SNORA1
Looking for SNORA12
Looking for SNORA14B
Looking for SNORA16A
Looking for SNORA21
Looking for SNORA23
Looking for SNORA24
Looking for SNORA32
Looking for SNORA38
Looking for SNORA44
Looking for SNORA48
Looking for SNORA49
Looking for SNORA52
Looking for SNORA53
Looking for SNORA57
Looking for SNORA61
Looking for SNORA63
Looking for SNORA64
Looking for SNORA65
Looking for SNORA70
Looking for SNORA71C
Looking for SNORA73A
Looking for SNORA73B
Looking for SNORA75
Looking for SNORA78
Looking for SNORA8
Looking for SNORD10
Looking for SNORD102
Looking for SNORD104
Looking for SNORD105
Looking for SNORD105B
Looking for SNORD108
Looking for SNORD110
Looking for SNORD111B
Looking for SNORD113-3
Looking for SNORD113-4
Looking for SNORD114-1
Looking for SNORD114-13
Looking for SNORD114-14
Looking for SNORD114-19
Looking for SNORD114-20
Looking for SNORD114-21
Looking for SNORD115-23
Looking for SNORD115-32
Looking for SNORD116-13
Looking for SNORD119
Looking for SNORD12
Lookin

Check the isoforms for each gene

In [5]:
from pprint import pprint

pprint(isoforms)

{'AP1G1': ['212193', '212194', '212195', '212196'],
 'CDKN2B-AS1': [],
 'CFDP1': ['147441', '147442'],
 'CHD8': ['219633', '219634', '219635'],
 'CWF19L1': ['221922', '221923'],
 'DDX39B': ['195345', '195346', '195347', '195348', '195349', '195350'],
 'EEF2': ['148263',
          '162163',
          '162164',
          '162165',
          '162166',
          '162167',
          '162168',
          '162169',
          '162170',
          '162171',
          '162172',
          '162173',
          '162174',
          '162175'],
 'EIF4A1': ['222495'],
 'EIF4A2': ['188715',
            '188716',
            '188717',
            '188718',
            '188719',
            '188720',
            '188721'],
 'EIF4G2': ['154974',
            '154975',
            '154976',
            '154977',
            '154978',
            '154979',
            '154980',
            '154981',
            '154982',
            '154983',
            '154984',
            '154985',
            '154986',
    

Count how many isoforms there are for each gene

In [9]:
isoforms_count = {}
isoform_zero = set()
total_count = 0

for gene in isoforms:
    isoforms_count[gene] = len(isoforms[gene])
    if len(isoforms[gene]) == 0:
        isoform_zero.add(gene)
    total_count += len(isoforms[gene])

pprint(isoforms_count)
print(isoform_zero)
print(len(isoform_zero))
total_count

{'AP1G1': 4,
 'CDKN2B-AS1': 0,
 'CFDP1': 2,
 'CHD8': 3,
 'CWF19L1': 2,
 'DDX39B': 6,
 'EEF2': 14,
 'EIF4A1': 1,
 'EIF4A2': 7,
 'EIF4G2': 17,
 'EP400': 2,
 'GAS5': 3,
 'GNL3': 6,
 'HIF1A-AS2': 0,
 'HSPA8': 10,
 'HSPA9': 6,
 'IPO7': 6,
 'MEG8': 1,
 'MYRIP': 7,
 'NAN': 0,
 'NCL': 23,
 'NFATC3': 7,
 'NOP56': 2,
 'PCAT4': 0,
 'PPAN': 3,
 'PRKAA1': 3,
 'PRRC2A': 8,
 'PTCD3': 3,
 'RABGGTB': 5,
 'RACK1': 0,
 'RNF149': 3,
 'RPL10': 6,
 'RPL12': 3,
 'RPL13': 6,
 'RPL13A': 2,
 'RPL17': 4,
 'RPL21': 6,
 'RPL23': 0,
 'RPL23A': 3,
 'RPL4': 3,
 'RPL7A': 8,
 'RPLP2': 4,
 'RPS11': 0,
 'RPS13': 2,
 'RPS2': 0,
 'RPS20': 4,
 'RPS3': 0,
 'RPS8': 4,
 'SCARNA12': 2,
 'SF3B3': 3,
 'SLC25A3': 0,
 'SNHG1': 4,
 'SNHG12': 1,
 'SNHG20': 0,
 'SNHG3': 5,
 'SNHG5': 1,
 'SNHG6': 1,
 'SNHG7': 1,
 'SNHG8': 2,
 'SNHG9': 4,
 'SNORA1': 1,
 'SNORA12': 1,
 'SNORA14B': 2,
 'SNORA16A': 1,
 'SNORA21': 1,
 'SNORA23': 1,
 'SNORA24': 1,
 'SNORA32': 1,
 'SNORA38': 1,
 'SNORA44': 1,
 'SNORA48': 1,
 'SNORA49': 1,
 'SNORA52': 1,
 'SNO

396